In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier



In [55]:
dt = pd.read_csv('../train.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
def data_cl(dt):
    dt = dt.drop("CUS_ID" ,axis=1)
    
    ohi = pd.get_dummies(dt["GENDER"])
    dt = dt.drop("GENDER",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    ohi = pd.get_dummies(dt["AGE"])
    dt = dt.drop("AGE",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    
    
#     sizemap = {
#         "低":1,
#         "中":3,
#         "中高":5,
#         "高":7
#     }

#     dt["AGE"] = dt["AGE"].map(sizemap)
    
#     for i in dt.keys():
#         if dt[i].isnull().sum()>60000:
#             dt = dt.drop(i,axis=1)

    

    labelencoder = LabelEncoder()
    for i in dt.keys():    
        for a in dt[i]:
            if  a is not None and isinstance(a,str):
                dt[i] = labelencoder.fit_transform(dt[i].astype(str))
                break
    
    for i in dt.keys():
        dt[i] = dt[i].fillna(dt[i].mean())
    
    return dt    
    

In [57]:
dt = data_cl(dt)

In [58]:
fix_dt = dt.sample(frac = 1)

fraud_dt = dt.loc[dt['Y1'] == 1]
non_fraud_dt = fix_dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dtf = normal_distributed_dt.sample(frac=1)
# new_dt = normal_distributed_dt.sample(frac=1, random_state=42)

new_dtf.shape

(4000, 135)

In [59]:
xd = new_dtf.drop("Y1",axis=1)
yd = new_dtf["Y1"]

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xd,yd, test_size=0.3, random_state=0)

In [61]:
tree = DecisionTreeClassifier(criterion = 'entropy', random_state=0)

In [62]:
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [63]:
test_y_predicted = tree.predict(xd)

from sklearn import  ensemble, preprocessing, metrics
# 績效
fpr, tpr, thresholds = metrics.roc_curve(yd, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.89725


In [64]:
test_y_predicted = tree.predict(X_train)

from sklearn import  ensemble, preprocessing, metrics
# 績效
fpr, tpr, thresholds = metrics.roc_curve(y_train, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

1.0


In [65]:
df = pd.read_csv('../test.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
df = data_cl(df)

In [67]:
test_y_predicted = tree.predict(df)

In [68]:
fin = pd.read_csv('../submit_test.csv' , encoding='big5')
fin["Ypred"] = test_y_predicted
fin.to_csv('submit_test.csv')

In [69]:
test_y_predicted.sum()

50885